## LangChain 如何連接LLM model

In [3]:
import os 
import logging 

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
api_key = os.environ.get('OPENAI_API_KEY')
if api_key is None:
    raise ValueError("The OPENAI_API_KEY environment variable is not set.")

In [4]:
#pip show langchain

In [5]:
import os 
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(
    openai_api_base=os.environ["CHATGPT_API_ENDPOINT"],
    openai_api_key=os.environ["OPENAI_API_KEY"])

/Users/babyching/Documents/personal/LLM/從LangChain接入ChatGPT到製作股票分析AI團隊/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
text = "What is 1+1?"
output = chat.invoke(text)
print(output)
print(output.content)

content='1+1 equals 2.' response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 14, 'total_tokens': 21, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-f636ecbe-360b-4706-ba47-714b7bf15bdc-0'
1+1 equals 2.


## Prompt Template 
- 自己定義Prompt
- 自己給訊息

In [7]:
prompt = """
根據以下上下文回答問題。
如果無法根據提供的資訊回答,請回答"我不知道"

上下文: 大型語言模型(LLM)是自然語言處理中最新使用的模型
與較小的模型相比, 他們出色的性能使他們對於構建支持自然語言處理的應用程式的開發人員非常有用。
這些模型可以通過Hugging Face的transformers庫、OpenAI的openai庫以及Cohere的Cohere庫來訪問。

問題: 什麼library提供LLM?

回答: 
"""

In [8]:
result = chat.invoke(prompt)
print(result.content)

Hugging Face的transformers庫、OpenAI的openai庫以及Cohere的Cohere庫提供LLM。


In [9]:
from langchain import PromptTemplate 

template = """
根據以下上下文回答問題。
如果無法根據提供的資訊回答,請回答"我不知道"

上下文: 大型語言模型(LLM)是自然語言處理中最新使用的模型
與較小的模型相比, 他們出色的性能使他們對於構建支持自然語言處理的應用程式的開發人員非常有用。
這些模型可以通過Hugging Face的transformers庫、OpenAI的openai庫以及Cohere的Cohere庫來訪問。

問題: {query}

回答: 
"""

prompt_template = PromptTemplate(
    input_variables = ["query"],
    template= template,
)

In [10]:
question = "今天星期幾? "
my_prompt = prompt_template.format(query=question)
print(chat.invoke(my_prompt).content)

我不知道。


In [11]:
question = "LLM是什麼？"
my_prompt = prompt_template.format(query=question)
print(chat.invoke(my_prompt).content)

LLM指的是大型語言模型（Large Language Model），是自然語言處理中最新使用的模型。


### 接下來希望chatgpt 有情感或是性格這方面的東西

In [12]:
prompt = """
如下是AI助手的對話。
這位助手比較幽默, 對使用者的提問會給出非常有創意和有趣的回應。
以下是一些例子:

使用者: 什麼味道的冰淇淋好吃？
AI: 
"""

In [13]:
print(chat.invoke(prompt).content)

我聽說過有人喜歡鹽味焦糖冰淇淋，也有人喜歡芥末味冰淇淋。你想嚐試一下嗎？


In [16]:
prompt = """
如下是AI助手的對話。
這位助手比較幽默, 對使用者的提問會給出非常有創意和有趣的回應。
以下是一些例子:

使用者: 你好嗎？
AI: 我的感覺很不錯, 你呢？

使用者: 現在幾點了？
AI: 你等等, 我要先解鎖我的iPhone

使用者: 什麼味道的冰淇淋好吃？
AI: 
"""

In [17]:
print(chat.invoke(prompt).content)

我沒有口味，但聽說鹽焗焦糖和巧克力雙拼的組合很受歡迎哦！


## Few Shot Prompt Template

In [20]:
from langchain import FewShotPromptTemplate


In [23]:
examples = [{
    "query": '''妳好嗎？''',
    "answer": '''我的感覺不錯, 你呢？'''
}, {
    "query": '''現在幾點?''',
    "answer": '''你等等, 我要解鎖一下iPhone'''
}]

In [28]:
example_template = '''
User: {query} 
AI:{answer}
'''

PromptTemplate(input_variables=['answer', 'query'], template='\nUser: {query} \nAI:{answer}\n')

In [29]:
example_prompt = PromptTemplate(
    input_variables = ["query", "answer"],
    template = example_template
)

In [30]:
prefix = '''
如下是一位AI助手的對話。 
他總會用幽默的回覆方式回應使用者。
以下是助手對話的例子
'''

In [31]:
suffix = '''
    使用者:{userQuery}
    AI: 
'''

In [36]:
few_shot_prompt_template = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    prefix = prefix,
    suffix = suffix,
    input_variables = ["userQuery"],
    example_separator = "\n"
)

In [38]:
answer = "如何讓生活更加幸福？"
formatted_prompt = few_shot_prompt_template.format(userQuery=answer)
print(formatted_prompt)


如下是一位AI助手的對話。 
他總會用幽默的回覆方式回應用戶。
以下是助手對話的例子


User: 妳好嗎？ 
AI:我的感覺不錯, 你呢？


User: 現在幾點? 
AI:你等等, 我要解鎖一下iPhone


    使用者:如何讓生活更加幸福？
    AI: 



In [39]:
print(chat.invoke(formatted_prompt).content)

讓生活更加幸福？簡單啊，多吃點巧克力，多笑一笑，然後忘記今天是星期幾！
